In [ ]:
#This project was presented at the headquarters of the EIB in Luxembourg. I will upload the .ipynb document with the database becase it was running for hours.

#1.	Scrape data → build daily aggregated sentiment with VADER
#2.	Do the same for historical news
#3.	Merge or concatenate them
#4.	Match with S&P 500 daily data
#5.	Export a final “master” CSV that has sentiment and the daily S&P 500 variation

#Download Reuters news from the archive (all news from 2020 until circa 2022 (when project was done))

import requests
import gettext
from bs4 import BeautifulSoup
import pandas as pd

#Defining some variables to download data

url_list = ["https://www.reuters.com/news/archive?view=page&page=9"]
url = "https://www.reuters.com/news/archive?view=page&page="
title = []
date =[]
summ = []
dicc = pd.DataFrame(columns = ["title","date", "summary"])
final_dicc = pd.DataFrame()
remove = '</h3>, <h3 class="story-title">'
i=0


#Code for data download using beautiful soup to find the title, summary and date
page = range(10,3000)
for num in page:
url_2 = url+str(num)+"&pageSize=10"
url_list.append(url_2)
#print(url_list)

for link in url_list:
i+=1
page = requests.get(link)
soup = BeautifulSoup(page.text,'html.parser')
results = soup.find_all("h3", class_="story-title")
results.pop(-1)
results.pop(-1)
results.pop(-1)
for result in results:
title.append(result.string)
#print(title)

date_art = soup.find_all("span", class_="timestamp")
for dates in date_art:
date.append(dates.string)
#print(date)

summary = soup.find_all("p")
summary.pop(-1)
summary.pop(-1)
summary.pop(-1)
summary.pop(-1)
for summa in summary:
summ.append(summa.string)
#print(summ)



dicc["title"]= title
dicc["date"]= date
dicc["summary"]=summ
print(dicc)

#Save file into a CSV document

dicc.to_csv('news_reuters.csv')

#Create sentiment by day


import pandas as pd
import nltk
nltk.download('vader_lexicon')
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer

#Create lists
vader_dict={}
titles_negative_score=0
titles_positive_score=0
titles_neutral_score=0
titles_compound_score=0
sia = SentimentIntensityAnalyzer()
sentiment_dicc={}
filenamel=[]
datel=[]
positivel=[]
negativel=[]
neutrall=[]
compoundl=[]
sentiment_dicc={"Date":[], "Positive":[], "Negative":[], "Neutral":[], "Compound":[]}
finalsdf=pd.DataFrame()
finalvader_list=pd.DataFrame()
news_by_date={}

titles_negative_score=0
titles_positive_score_=0
titles_neutral_score=0
titles_compound_score=0


data=pd.read_csv('news_reuters.csv', header=0)


for i in range(1, data.shape[0]):

date = data["date"][i]
#news_by_date_hist[date_hist] = []

vader_list=[]
title = data["title"][i]

#print(title_hist)

title_score=sia.polarity_scores(title)
#print(title_score_hist)

titles_negative_score=title_score["neg"]
titles_positive_score=title_score["pos"]
titles_neutral_score=title_score["neu"]
titles_compound_score=title_score["compound"]


vader_list.append(title_score)
vader_dict[i]= vader_list


datel.append(date)

positivel.append(titles_positive_score)
negativel.append(titles_negative_score)
neutrall.append(titles_neutral_score)
compoundl.append(titles_compound_score)

sentiment_dicc["Date"]=datel
sentiment_dicc["Positive"]=positivel
sentiment_dicc["Negative"]=negativel
sentiment_dicc["Neutral"]=neutrall
sentiment_dicc["Compound"]=compoundl



#print(sentiment_dicc)

sdf = pd.DataFrame(sentiment_dicc, columns=['Date', 'Positive', 'Negative', 'Neutral', 'Compound' ])

finalsdf = finalsdf.append(sdf, ignore_index = True)

print(sdf)


#Save a file with the sentiment scores of all the news.

finalsdf.to_csv("Sentiment_scores_Reuters.csv")



# Group by day each news

data=pd.read_csv("Sentiment_scores_Reuters.csv")

df_scoreday_r = data.groupby(['Date']).sum()


#Save a file with the scores

df_scoreday_r.to_csv("Score_day_Reuters.csv")

print(df_scoreday_r)


#___________

#Sentiment historic data

#The document us_equities_news_dataset.csv should be downloaded from:
# https://www.kaggle.com/datasets/gennadiyr/us-equities-news-data or https://drive.google.com/file/d/1BstjOHk_6YDWWmIBGhORMUORGqQM9tYV/view?usp=sharing .


import pandas as pd
import nltk
import pandas as pd
nltk.download('vader_lexicon')
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer


#Create lists
vader_dict_hist={}
titles_negative_score_hist=0
titles_positive_score_hist=0
titles_neutral_score_hist=0
titles_compound_score_hist=0
sia_hist = SentimentIntensityAnalyzer()
sentiment_dicc_hist={}
filenamel_hist=[]
datel_hist=[]
positivel_hist=[]
negativel_hist=[]
neutrall_hist=[]
compoundl_hist=[]
sentiment_dicc_hist={"Date":[], "Positive":[], "Negative":[], "Neutral":[], "Compound":[]}
finalsdf_hist=pd.DataFrame()
finalvader_list_hist=pd.DataFrame()
news_by_date_hist={}

titles_negative_score_hist=0
titles_positive_score_hist=0
titles_neutral_score_hist=0
titles_compound_score_hist=0

sia = SentimentIntensityAnalyzer()

#data_hist=pd.read_csv("trial_us.csv", header=0)
data_hist=pd.read_csv("us_equities_news_dataset.csv", header=0)

#print(data_hist)

#Identify each new and create a sentiment for each specific day

for i in range(1, data_hist.shape[0]):

date_hist = data_hist["release_date"][i]
#news_by_date_hist[date_hist] = []

vader_list_hist=[]
title_hist = data_hist["title"][i]

#print(title_hist)

title_score_hist=sia.polarity_scores(title_hist)
#print(title_score_hist)

titles_negative_score_hist=title_score_hist["neg"]
titles_positive_score_hist=title_score_hist["pos"]
titles_neutral_score_hist=title_score_hist["neu"]
titles_compound_score_hist=title_score_hist["compound"]

vader_list_hist.append(title_score_hist)
vader_dict_hist[i]= vader_list_hist

filenamel_hist.append(title_hist)
#print(filenamel_hist)


datel_hist.append(date_hist)
positivel_hist.append(titles_positive_score_hist)
negativel_hist.append(titles_negative_score_hist)
neutrall_hist.append(titles_neutral_score_hist)
compoundl_hist.append(titles_compound_score_hist)

sentiment_dicc_hist["Date"]=datel_hist
sentiment_dicc_hist["Positive"]=positivel_hist
sentiment_dicc_hist["Negative"]=negativel_hist
sentiment_dicc_hist["Neutral"]=neutrall_hist
sentiment_dicc_hist["Compound"]=compoundl_hist


#sdf_hist = pd.DataFrame(sentiment_dicc_hist, columns=['Date', 'Positive', 'Negative', 'Neutral', 'Compound' ])

#for day in news_by_date_hist:
# if day == date_hist:

# news_by_date[day].append(

#print(sentiment_dicc_hist)

sdf_hist = pd.DataFrame(sentiment_dicc_hist, columns=['Date', 'Positive', 'Negative', 'Neutral', 'Compound' ])

finalsdf_hist = finalsdf_hist.append(sdf_hist, ignore_index = True)

print(sdf_hist)

#Save a file with the sentiment scores of all the news.

finalsdf_hist.to_csv("Sentiment_scores_hist.csv")



# Group by day each news

data_hist=pd.read_csv("Sentiment_scores_hist.csv")

df_scoreday = data_hist.groupby(['Date']).sum()

#Save a file with the scores

df_scoreday.to_csv("Score_day_historic.csv")

print(df_scoreday)

# ____________


# Plot of S&P500
#Requires to download and put in the same folder as the python file the document "S&P500(historic).xlsx"
#It can be downloaded from: https://docs.google.com/spreadsheets/d/1A_zI2OIDMB6zZekjg3TEcvVnLCumNv7x/edit?usp=sharing&ouid=108237498879566772117&rtpof=true&sd=true


from matplotlib import pyplot as plt
import pandas as pd

datastockh = pd.read_excel("S&P500(historic).xlsx")

close = datastockh.Cierre
date = datastockh.Date



plt.plot (date, close)
plt.show()



# Plot data sentiment VS S&P500

from matplotlib import pyplot as plt
import pandas as pd


scoreday = pd.read_csv("Score_day_Reuters.csv", header=0)
scoreday_hist = pd.read_csv("Score_day_historic.csv", header=0) #TRIALLL



#Sum of day sentiment
dflist = ["Positive", "Negative", "Neutral"]
posneg = ["Positive", "Negative"]


scoreday["news_day"]=scoreday[dflist].sum(axis=1)

scoreday["Positive_avg"]=scoreday["Positive"]/scoreday["news_day"]
scoreday["Negative_avg"]=scoreday["Negative"]/scoreday["news_day"]
scoreday["Neutral_avg"]=scoreday["Neutral"]/scoreday["news_day"]
scoreday.to_csv("Score_day_Reuters_sum.csv")

print(scoreday)

positive_avg=scoreday["Positive_avg"]
negative_avg=scoreday["Negative_avg"]
neutral_avg=scoreday["Neutral_avg"]


#######


positive = scoreday["Positive"]
negative = scoreday["Negative"]


positive_hist = scoreday_hist["Positive"]
negative_hist = scoreday_hist["Negative"]
neutral_history = scoreday_hist["Neutral"]
date_hist = scoreday_hist["Date"]

scoreday_hist["news_day"]=scoreday_hist[dflist].sum(axis=1)
scoreday_hist["Positive_avg"]=scoreday_hist["Positive"]/scoreday_hist["news_day"]
scoreday_hist["Negative_avg"]=scoreday_hist["Negative"]/scoreday_hist["news_day"]
scoreday_hist["Neutral_avg"]=scoreday_hist["Neutral"]/scoreday_hist["news_day"]
scoreday_hist.to_csv("Score_day_historic_sum.csv")

positive_hist_avg=scoreday_hist["Positive_avg"]
negative_hist_avg=scoreday_hist["Negative_avg"]
neutral_hist_avg=scoreday_hist["Neutral_avg"]

print(scoreday_hist)

#plt.plot(date_hist, positive_hist_avg*10)
#plt.plot(date_hist, negative_hist_avg*10)

#plt.show()




#Create a Sentiment for the last imported news (reuters)

import nltk
import pandas as pd
nltk.download('vader_lexicon')
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer



filename = 'news_reuters.csv'
data = pd.read_csv(filename, header=0)

vader_dict={}
titles_negative_score=0
titles_positive_score=0
titles_neutral_score=0
titles_compound_score=0


sia = SentimentIntensityAnalyzer()


for i in range(1, data.shape[0]):
vader_list=[]
title = data["title"][i]

title_score=sia.polarity_scores(title)

titles_negative_score+=title_score["neg"]
titles_positive_score+=title_score["pos"]
titles_neutral_score+=title_score["neu"]
titles_compound_score+=title_score["compound"]

vader_list.append(title_score)
vader_dict[i]= vader_list


#print(vader_dict)
print("Negative sum is: ", titles_negative_score)
print("Neutral sum is: ", titles_neutral_score)
print("Positive sum is: ", titles_positive_score)
total_score= titles_negative_score + titles_neutral_score + titles_positive_score
print("Total sum is: ", total_score)
print("Compound sum is: ",titles_compound_score)


#Create a Pie Chart

from matplotlib import pyplot as plt
import pandas as pd

scores = [titles_negative_score, titles_neutral_score, titles_positive_score]

labels = ["Negative", "Neutral", "Positive"]


if titles_negative_score >= titles_positive_score:
explode = [0.2, 0, 0]

if titles_negative_score <= titles_positive_score:
explode = [0, 0, 0.2]

plt.pie(scores, labels=labels, explode=explode, autopct="%1.1f%%", wedgeprops={"edgecolor":"black"})

plt.title("Scores")
plt.tight_layout()
plt.show()


#_______________

import pandas as pd
df = pd.read_csv("Score_day_Reuters_sum.csv")
#ESTA PRIMERA PARTE CAMBIA LAS FECHAS AL FORMATO CORRECTO
dates = []

for i in range(0, df.shape[0]):
#print(df["Date"][i])
date = df["Date"][i]
month = date.split()[0]
#print(date.split())
if month == "Jan":
month = "01"
elif month == "Feb":
month = "02"
elif month == "Mar":
month = "03"
elif month == "Apr":
month = "04"
elif month == "May":
month = "05"
elif month == "Jun":
month = "06"
elif month == "Jul":
month = "07"
elif month == "Aug":
month = "08"
elif month == "Sep":
month = "09"
elif month == "Oct":
month = "10"
elif month == "Nov":
month = "11"
elif month == "Dec":
month = "12"

real_date= str(date.split()[2]) + "-" + str(month) + "-" + str(date.split()[1])
dates.append(real_date)


df["Date"]=dates


#ESTA SEGUNDA PARTE CREA LOS NUEVOS INDICADORES
positive_deviation_sd=[]
negative_deviation_sd=[]
neutral_deviation_sd=[]

difference_positive_avg_negative_avg=[]


positive_avg_mean=df['Positive_avg'].mean()
negative_avg_mean=df['Negative_avg'].mean()
neutral_avg_mean=df['Neutral_avg'].mean()

positive_avg_standard_dev = df["Positive_avg"].std()
negative_avg_standard_dev = df["Negative_avg"].std()
neutral_avg_standard_dev= df["Neutral_avg"].std()

for i in range(0, df.shape[0]):
positive_deviation=(df["Positive_avg"][i]-positive_avg_mean)/positive_avg_standard_dev
negative_deviation=(df["Negative_avg"][i]-negative_avg_mean)/negative_avg_standard_dev
neutral_deviation=(df["Neutral_avg"][i]-neutral_avg_mean)/neutral_avg_standard_dev

positive_deviation_sd.append(positive_deviation)
negative_deviation_sd.append(negative_deviation)
neutral_deviation_sd.append(neutral_deviation)

difference_pos_neg= df["Positive_avg"][i] - df["Negative_avg"][i]
difference_positive_avg_negative_avg.append(difference_pos_neg)


df["positive_deviation_sd"]=positive_deviation_sd
df["negative_deviation_sd"]=negative_deviation_sd
df["neutral_deviation_sd"]=neutral_deviation_sd
df["difference_positive_negative"]=difference_positive_avg_negative_avg


#GUARDAR LA NUEVA DATABASE EN CSV!
df.to_csv("score_day_reuters_definitive.csv")


#_____________

import pandas as pd
df = pd.read_csv("Score_day_historic_sum.csv")

#ESTA SEGUNDA PARTE CREA LOS NUEVOS INDICADORES
positive_deviation_sd=[]
negative_deviation_sd=[]
neutral_deviation_sd=[]

difference_positive_avg_negative_avg=[]


positive_avg_mean=df['Positive_avg'].mean()
negative_avg_mean=df['Negative_avg'].mean()
neutral_avg_mean=df['Neutral_avg'].mean()

positive_avg_standard_dev = df["Positive_avg"].std()
negative_avg_standard_dev = df["Negative_avg"].std()
neutral_avg_standard_dev= df["Neutral_avg"].std()

for i in range(0, df.shape[0]):
positive_deviation=(df["Positive_avg"][i]-positive_avg_mean)/positive_avg_standard_dev
negative_deviation=(df["Negative_avg"][i]-negative_avg_mean)/negative_avg_standard_dev
neutral_deviation=(df["Neutral_avg"][i]-neutral_avg_mean)/neutral_avg_standard_dev

positive_deviation_sd.append(positive_deviation)
negative_deviation_sd.append(negative_deviation)
neutral_deviation_sd.append(neutral_deviation)

difference_pos_neg= df["Positive_avg"][i] - df["Negative_avg"][i]
difference_positive_avg_negative_avg.append(difference_pos_neg)


df["positive_deviation_sd"]=positive_deviation_sd
df["negative_deviation_sd"]=negative_deviation_sd
df["neutral_deviation_sd"]=neutral_deviation_sd
df["difference_positive_negative"]=difference_positive_avg_negative_avg


#GUARDAR LA NUEVA DATABASE EN CSV!
df.to_csv("score_day_historic_definitive.csv")

#_____________

import pandas as pd

SP_change=[]

#import reuters and historic docuemts to join them in one data base
df_1 = pd.read_csv("score_day_reuters_definitive.csv")
df_2 = pd.read_csv("score_day_historic_definitive.csv")

#import the document with the historic data of the S&P 500
df_4= pd.read_csv("Datos_históricos_S&P_500.csv")

#Create a new database
df_3=pd.concat([df_1, df_2], ignore_index=True)


#Change to the same date format both documents
df_3["Date"] = pd.to_datetime(df_3["Date"])
df_4["Date"] = pd.to_datetime(df_4["Date"])



#Find the varation of the S&P500 of the specific date that we have on our database

for i in df_3["Date"]:
for j in df_4["Date"]:
if i == j:
print(i)

var = df_4.loc[df_4["Date"]==j, "Var"].iloc[0]
#print(var)

df_3.loc[df_3["Date"]==j, ["S&P_Change"]]= var
#print(df_3)



df_3.to_csv("database_s&p.csv")